In [4]:
import pandas as pd
import tqdm
import glob
import os

In [5]:
files = glob.glob("../data/wikipedia/a.parquet")
files = [f for f in files if "all" not in f]

In [6]:
chunk_size = 10000000

In [42]:
def output_seperated_files(token_length, stride, drop_category):    
    output_dir = f"../data/wikipedia/sep_chunk/token_length{token_length}_stride_sentence{stride}_drop_category{drop_category}"
    os.makedirs(output_dir, exist_ok=True)
    chunk = 1
    df_wiki_sep = []
    for file in files:
        df_wiki = pd.read_parquet(file)
        df_wiki = df_wiki.drop("categories", axis=1)
        for i in tqdm.tqdm(range(len(df_wiki))):
            series = df_wiki.iloc[i]
            text = series["text"]
            if drop_category:
                idx = text.find(" Category:")
                text = text[:idx]

            text_space = text.split(".")
            text = ""
            text_stride = []

            series_ = series.to_dict()
            series_["text"] = " ".join(text_space[:stride*2])
            series_["title_only"] = True
            df_wiki_sep.append(series_)
            
            appended = False
            for j in range(0, len(text_space)):
                series_ = series.to_dict()
                text += text_space[j] + "."
                text_stride.append(text_space[j])
                text_stride = text_stride[-stride:]
                appended = False
                if len(text.split(" ")) > token_length:
                    series_["text"] = text
                    series_["title_only"] = False
                    df_wiki_sep.append(series_)
                    text = ".".join(text_stride)
                    appended = True
            if not appended:
                series_["text"] = text
                series_["title_only"] = False
                df_wiki_sep.append(series_)

            if len(df_wiki_sep) > chunk_size:
                print(f"chunk {chunk}")
                df_wiki_sep = pd.DataFrame(df_wiki_sep)
                df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
                df_wiki_sep = []
                chunk += 1
    df_wiki_sep = pd.DataFrame(df_wiki_sep)
    df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
    

In [43]:
output_seperated_files(token_length=90, stride=3, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 442726/442726 [02:18<00:00, 3188.37it/s]


In [46]:
output_seperated_files(token_length=120, stride=4, drop_category=True)

100%|███████████████████████████████| 442726/442726 [02:27<00:00, 3007.77it/s]


In [47]:
output_seperated_files(token_length=150, stride=5, drop_category=True)

100%|███████████████████████████████| 442726/442726 [03:04<00:00, 2404.34it/s]


In [48]:
output_seperated_files(token_length=240, stride=8, drop_category=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 442726/442726 [02:45<00:00, 2680.31it/s]
